In [30]:
import pandas as pd
import mat73
from datetime import datetime
import numpy as np
from tqdm import tqdm
from utils import *
import h5py

In [2]:
df_brandon = pd.read_csv('work_tables/EEGs_And_Reports_20231024.csv')

In [3]:
df = pd.read_excel('tables/batch1.xlsx')
df = df.rename({'file':'HashFolderName'},axis=1)
df=df.merge(df_brandon[['HashFolderName','SiteID','SessionID_new','EEGFolder']],on='HashFolderName')

In [4]:
windowsize = 15
desired_channels= ['Fp1','F3','C3','P3','F7','T3','T5','O1','Fz','Cz','Pz','Fp2','F4','C4','P4','F8','T4','T6','O2']

In [38]:
# process dataframe
root = 'bdsp/opendata/EEG/data/'
HashFolderNames = df.HashFolderName.unique().tolist()
hf = h5py.File('SauronData.h5', 'w')

In [39]:
# process EEG file
for i in tqdm(range(10)):
    # get all events from same hashfolder
    sub_df = df[df.HashFolderName==HashFolderNames[i]]

    # get knowledge about file location
    SiteID,EEGFolder,HashFolderName = sub_df.iloc[0][['SiteID','EEGFolder','HashFolderName']].to_list()
    # load file and metadata
    data,Fs,channels,absolute_eeg_start_time = process_EEG_file(root,SiteID,EEGFolder,HashFolderName)
    
    # catch some potential problems
    if not 200 in Fs:
        print(f'frequency is {Fs}, instead of the normal 200!')
        break   
    if not set(desired_channels).issubset(channels):
        print(f'not all desired channels are present!\n want: {desired_channels}\n got: {channels}')
        break

    # save all events from the eeg file
    for j in range(len(sub_df)):
        # get time relative to beginning of file 
        absolute_event_time = sub_df.iloc[j].time
        relative_event_time = int((absolute_event_time-absolute_eeg_start_time).total_seconds())
    
        if relative_event_time>12*3600:
            break
        # get snippet from eeg file and save it somewhere
        snippet = get_event_snippet(data=data,
                                    relative_event_time=relative_event_time,
                                    windowsize=windowsize,
                                    all_channels=channels,
                                    desired_channels=desired_channels,
                                    Fs=Fs)
        name = HashFolderName+'_'+str(relative_event_time)+'.npy'
        # save in h5py file
        hf.create_dataset(name=name,data=snippet,dtype='f4',compression='gzip')

100%|██████████| 10/10 [00:05<00:00,  1.75it/s]
